# FAO Land Cover Classification

The purpose of this notebook is to provide an easy-to-use method for testing environmental layers to use for classification and seeing how changes to particular layers effect the final Land Cover Classification. You can easily test with different environmental layer inputs, and differnet locations. 

This code defines 5 variables to contain the binary layers required to reach a level 3 classification:
1. **vegetat_veg_cat:** Vegetated / Non-Vegetated 
2. **aquatic_wat_cat:** Water / Terrestrial 
3. **cultman_agr_cat:** Natural Veg / Crop or Managed Veg
4. **artific_urb_cat:** Natural Surfaces / Artificial Surfaces (bare soil/urban) 
5. **artwatr_wat_cat:** Natural water / Artificial water

Whilst this example is using open data cube to load the required data, it can be loaded from anywhere - so long as all input layers cover the same geographic region and are defined in a correctly labelled dataset, before being passed to the classification code.


In [ ]:
import numpy
import xarray
import scipy
from matplotlib import pyplot
from matplotlib import cm
import datacube
from datacube.storage import masking
dc = datacube.Datacube(app="le_lccs")

#import classification script
import lccs_l3

Define details of data to be loaded - area, resolution, crs..

In [ ]:
# Define area of interest

# Ayr
x = (1500000, 1600000)
y = (-2200000, -2100000)

# # Diamentina
#x = (800000, 900000)
#y = (-2800000, -2700000)

# # Gwydir
#x = (1600000, 1700000)
#y = (-3400000, -3300000)

# Leichhardt
#x = (800000, 900000)
#y = (-2000000, -1900000)

# # Barmah-Millewa
#x = (1100000, 1200000)
#y = (-4000000, -3900000)

# # Forescue marshes
#x = (-1200000, -1300000)
#y = (-2500000, -2400000)

# # Snowy
#x = (1400000, 1500000)
#y = (-4100000, -4000000)

res = (-25, 25)
crs = "EPSG:3577"
time = ("2010-01-01", "2010-12-15")
sensor= 'ls5'
query=({'x':x,
        'y':y,
        'crs':crs,
        'resolution':res})

out_filename = "Townsville-2010.tif"

## Create environmental layers

### Presence/Absence of Vegetation

INITIAL-LEVEL DISTINCTION

   * *Primarily Vegetated Areas*:
   This class applies to areas that have a vegetative cover of at least 4% for at least two months of the year, consisting of Woody (Trees, Shrubs) and/or Herbaceous (Forbs, Graminoids) lifeforms, or at least 25% cover of Lichens/Mosses when other life forms are absent.
   * *Primarily Non-Vegetated Areas*:
   Areas which are not primarily vegetated.
   
   ****LINK to info on this

 ### <font color='red'>Explain what we used and why... </font>
 ****LINK to papers as required


**Creating your own layer**: To use a different veg/non-veg layer, replace the following two cells with code to create a binary layer with vegetated (1) and non-vegetated (0), using the method of choice, and save into `vegetat_veg_cat_ds`

In [ ]:
# Load data from datacube
fc_ann = dc.load(product="fc_percentile_albers_annual", 
                 measurements=["PV_PC_50", "NPV_PC_50"],
                 time=time, **query)

fc_ann = masking.mask_invalid_data(fc_ann)

In [ ]:
# Create binary layer representing vegetated (1) and non-vegetated (0)
vegetat = ((fc_ann["PV_PC_50"] >= 55) | (fc_ann["NPV_PC_50"] >= 55))

# Convert to Dataset and add name
vegetat_veg_cat_ds = vegetat.to_dataset(name="vegetat_veg_cat").squeeze().drop('time')

In [ ]:
# # Plot output
# vegetat_veg_cat_ds["vegetat_veg_cat"].plot(figsize=(6, 5))

### Aquatic or regularly flooded / Terrestrial

SECOND-LEVEL DISTINCTION
This layer breaks the initial veg/non-veg into 4 classes based on the presence or absence of water

   * *Primarily vegetated, Terrestrial*: The vegetation is influenced by the edaphic substratum
   * *Primarily Non-Vegetated, Terrestrial*: The cover is influenced by the edaphic substratum
   * *Primarily vegetated, Aquatic or regularly flooded*: The environment is significantly influenced by the presence of water over extensive periods of time. The water is the dominant factor determining natural soil development and the type of plant communities living on its surface
   * *Primarily Non-Vegetated, Aquatic or regularly flooded*:
   
   
   ****LINK to info on this

### <font color='red'>Explain what we used and why... </font>
****LINK to papers as required


**Creating your own layer**: To use a different veg/non-veg layer, replace the following two cells with code to create a binary layer with aquatic (1) and terrestrial (0), using the method of choice, and save into `aquatic_wat_cat_ds`

In [ ]:
# Load data from datacube
wofs_ann = dc.load(product="wofs_annual_summary", measurements=["frequency"], 
                     time=time, **query)

wofs_ann = masking.mask_invalid_data(wofs_ann)

In [ ]:
# Create binary layer representing aquatic (1) and terrestrial (0)
aquatic_wat = ((wofs_ann["frequency"] >= 0.01))

# Convert to Dataset and add name
aquatic_wat_cat_ds = aquatic_wat.to_dataset(name="aquatic_wat_cat").squeeze().drop('time')

In [ ]:
# # Plot output
# aquatic_wat_cat_ds["aquatic_wat_cat"].plot(figsize=(6, 5))

### cultman_agr_cat

In [ ]:
# Load data from datacube
ls8_mads = dc.load(product=sensor +"_nbart_tmad_annual", time=time, **query)
ls8_mads = masking.mask_invalid_data(ls8_mads)

In [ ]:
# Create binary layer representing cultivated (1) and natural (0)
cultman = ((ls8_mads["edev"] > 0.115))

# Convert to Dataset and add name
cultman_agr_cat_ds = cultman.to_dataset(name="cultman_agr_cat").squeeze().drop('time')

In [ ]:
# # Plot output
# cultman_agr_cat_ds["cultman_agr_cat"].plot(figsize=(6, 5))

### artific_urb_cat

In [ ]:
# Load data
ls8_gm = dc.load(product= sensor + "_nbart_geomedian_annual", time=time, **query)
ls8_gm = masking.mask_invalid_data(ls8_gm).squeeze().drop('time')

In [ ]:
# Calculate ndvi
ndvi = ((ls8_gm.nir - ls8_gm.red) / (ls8_gm.nir + ls8_gm.red))

# Calculate NDBI
NDBI = ((ls8_gm.nir - ls8_gm.swir1) /  (ls8_gm.nir + ls8_gm.swir1))

In [ ]:
# Create binary layer representing urban (1) and baresoil (0)
urban = (NDBI.where(ndvi<0.15) < 0)

# Convert to Dataset and add name
artific_urb_cat = urban.to_dataset(name="artific_urb_cat")

In [ ]:
# # Plot output
# artific_urb_cat["artific_urb_cat"].plot(figsize=(6, 5))

### artwatr_wat_cat

In [ ]:
# Load data
geofab = dc.load(product="geofabric",measurements=["band1"], **query)
geofab = geofab.squeeze().drop('time')

In [ ]:
# # Plot data
# geofab.band1.plot.imshow(cmap="nipy_spectral")

In [ ]:
# Create binary layer representing artificial water (1) and natural water (0)
artwatr_wat_cat_ds = ((geofab["band1"] == 1) | (geofab["band1"] == 8))

In [ ]:
# Convert to Dataset and add name
artwatr_wat_cat_ds = artwatr_wat_cat_ds.to_dataset(name="artwatr_wat_cat")

In [ ]:
# # Plot output
# artwatr_wat_cat_ds["artwatr_wat_cat"].plot(figsize=(5, 5))

Collect environmental variables into array for passing to classification system

In [ ]:
variables_xarray_list = []
variables_xarray_list.append(artwatr_wat_cat_ds)
variables_xarray_list.append(aquatic_wat_cat_ds)
variables_xarray_list.append(vegetat_veg_cat_ds)
variables_xarray_list.append(cultman_agr_cat)
variables_xarray_list.append(artific_urb_cat)

## Classification

The LCCS classificaition is hierachial. The 8 classes are shown below.

| Class name | Code | Numeric code |
|----------------------------------|-----|-----|
| Cultivated Terrestrial Vegetated | A11 | 111 |
| Natural Terrestrial Vegetated | A12 | 112 |
| Cultivated Aquatic Vegetated | A23 | 123 |
| Natural Aquatic Vegetated | A24 | 124 |
| Artificial Surface | B15 | 215 |
| Natural Surface | B16 | 216 |
| Artificial Water | B27 | 227 |
| Natural Water | B28 | 228 |


In [ ]:
# Merge to a single dataframe
classification_data = xarray.merge(variables_xarray_list)
#classification_data

In [ ]:
# Apply Level 3 classification using separate function. Works through in three stages
level1, level2, level3 = lccs_l3.classify_lccs_level3(classification_data)

In [ ]:
# Save classification values back to xarray
out_class_xarray = xarray.Dataset(
    {"level1" : (classification_data["vegetat_veg_cat"].dims, level1),
     "level2" : (classification_data["vegetat_veg_cat"].dims, level2),
     "level3" : (classification_data["vegetat_veg_cat"].dims, level3)})
classification_data = xarray.merge([classification_data, out_class_xarray])

col_level2 = cm.get_cmap("Set1", 2)
# classification_data.level2.plot(cmap=(col_level2))

In [ ]:
# print("level 1:",numpy.unique(classification_data.level1))
# print("level 2:",numpy.unique(classification_data.level2))
# print("level 3:",numpy.unique(classification_data.level3))

In [ ]:
#To check the results for level 3 use colour_lccs_level3 to get the colour scheme.
pyplot.figure(figsize=(10, 10))
red, green, blue, alpha = lccs_l3.colour_lccs_level3(level3)
pyplot.imshow(numpy.dstack([red, green, blue, alpha]))

### Save results to geotiff

In [ ]:
import gdal
def array_to_geotiff(fname, data, geo_transform, projection,
                     nodata_val=0, dtype=gdal.GDT_Int16):

    # Set up driver
    driver = gdal.GetDriverByName('GTiff')

    # Create raster of given size and projection
    rows, cols = data.shape
    dataset = driver.Create(fname, cols, rows, 1, dtype)
    dataset.SetGeoTransform(geo_transform)
    dataset.SetProjection(projection)

    # Write data to array and set nodata values
    band = dataset.GetRasterBand(1)
    band.WriteArray(data)
    band.SetNoDataValue(nodata_val)

    # Close file
    dataset = None